In [1]:
import pandas as pd
import numpy as np

In [76]:
champs = pd.read_csv("champs.csv")
matches = pd.read_csv("matches.csv")
participants = pd.read_csv("participants.csv")
stats1 = pd.read_csv("stats1.csv")
stats2 = pd.read_csv("stats2.csv")
teambans = pd.read_csv("teambans.csv")
teamstats = pd.read_csv("teamstats.csv")

C:\Users\Kiki\AppData\Local\Temp\ipykernel_15476\4269832138.py:5: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  stats2 = pd.read_csv("stats2.csv")


In [77]:
#champs.info()
matches.info()
#teamstats.info()
#stats1.info()
#stats2.info()
#teambans.info()
#participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184069 entries, 0 to 184068
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          184069 non-null  int64 
 1   gameid      184069 non-null  int64 
 2   platformid  184069 non-null  object
 3   queueid     184069 non-null  int64 
 4   seasonid    184069 non-null  int64 
 5   duration    184069 non-null  int64 
 6   creation    184069 non-null  int64 
 7   version     184069 non-null  object
dtypes: int64(6), object(2)
memory usage: 11.2+ MB


In [78]:
# Izračunavamo broj igrača po meču
player_count = participants.groupby('matchid').size().reset_index(name='player_count')

print(player_count)

        matchid  player_count
0            10            10
1            11            10
2            12            10
3            13            10
4            14            10
...         ...           ...
184064   187584            10
184065   187585            10
184066   187586            10
184067   187587            10
184068   187588            10

[184069 rows x 2 columns]


In [79]:
player_count_less_than_10 = player_count[player_count['player_count'] < 10]

print(player_count_less_than_10)

        matchid  player_count
121453   124937             6
121561   125045             6
121562   125046             6
121563   125047             6
121564   125048             6
...         ...           ...
183476   186996             6
183477   186997             6
183478   186998             6
183479   186999             6
183480   187000             6

[1541 rows x 2 columns]


In [80]:
# Izdvajamo listu mečeva koji imaju manje od 10 igrača
matches_to_delete = player_count_less_than_10['matchid']

print(matches_to_delete)

121453    124937
121561    125045
121562    125046
121563    125047
121564    125048
           ...  
183476    186996
183477    186997
183478    186998
183479    186999
183480    187000
Name: matchid, Length: 1541, dtype: int64


In [81]:
id_to_matchid = participants.set_index('id')['matchid'].to_dict()

# Dodajte matchid kolonu u stats1 i stats2 koristeći mapu
stats1['matchid'] = stats1['id'].map(id_to_matchid)
stats2['matchid'] = stats2['id'].map(id_to_matchid)

player_count = participants['matchid'].value_counts()
matches_with_less_than_10_players = player_count[player_count < 10].index

In [82]:
player_count = participants['matchid'].value_counts()
matches_with_less_than_10_players = player_count[player_count < 10].index

print(f"Broj mečeva sa manje od 10 igrača: {len(matches_with_less_than_10_players)}")

# Filtrirajte mečeve iz stats1 i stats2 koji nemaju statistike za svih 10 igrača
stats1_cleaned = stats1[~stats1['matchid'].isin(matches_with_less_than_10_players)]
stats2_cleaned = stats2[~stats2['matchid'].isin(matches_with_less_than_10_players)]

# Proverite koliko redova je obrisano
print(f"Broj redova pre čišćenja u stats1: {len(stats1)}")
print(f"Broj redova posle čišćenja u stats1: {len(stats1_cleaned)}")
print(f"Broj redova pre čišćenja u stats2: {len(stats2)}")
print(f"Broj redova posle čišćenja u stats2: {len(stats2_cleaned)}")

Broj mečeva sa manje od 10 igrača: 1541
Broj redova pre čišćenja u stats1: 999999
Broj redova posle čišćenja u stats1: 999999
Broj redova pre čišćenja u stats2: 834518
Broj redova posle čišćenja u stats2: 825280


In [83]:
#Brisanje nedostajućih podataka
matches = matches[~matches['id'].isin(matches_with_less_than_10_players)]
participants = participants[~participants['matchid'].isin(matches_with_less_than_10_players)]
#stats1 = stats1[stats1['id'].isin(participants['id'])]
#stats2 = stats2[stats2['id'].isin(participants['id'])]
teambans = teambans[~teambans['matchid'].isin(matches_with_less_than_10_players)]
teamstats = teamstats[~teamstats['matchid'].isin(matches_with_less_than_10_players)]

In [84]:
"""matches.to_csv("clean_matches.csv", index=False)
participants.to_csv("clean_participants.csv", index=False)
teambans.to_csv("clean_teambans.csv", index=False)
teamstats.to_csv("clean_teamstats.csv", index=False)"""

'matches.to_csv("clean_matches.csv", index=False)\nparticipants.to_csv("clean_participants.csv", index=False)\nteambans.to_csv("clean_teambans.csv", index=False)\nteamstats.to_csv("clean_teamstats.csv", index=False)'

In [85]:
stats_combined = pd.concat([stats1_cleaned, stats2_cleaned])

In [86]:
stats_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825279 entries, 0 to 834517
Data columns (total 57 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   id                      int64 
 1   win                     int64 
 2   item1                   int64 
 3   item2                   int64 
 4   item3                   int64 
 5   item4                   int64 
 6   item5                   int64 
 7   item6                   int64 
 8   trinket                 int64 
 9   kills                   int64 
 10  deaths                  int64 
 11  assists                 int64 
 12  largestkillingspree     int64 
 13  largestmultikill        int64 
 14  killingsprees           int64 
 15  longesttimespentliving  int64 
 16  doublekills             int64 
 17  triplekills             int64 
 18  quadrakills             int64 
 19  pentakills              int64 
 20  legendarykills          int64 
 21  totdmgdealt             int64 
 22  magicdmgdealt      

In [87]:
# Merge participants with combined stats on id
merged = participants.merge(stats_combined, how='left', on='id', indicator=True)

# Filter rows where merge indicator is 'left_only' (indicating missing stats)
missing_stats = merged[merged['_merge'] == 'left_only']
print(missing_stats)

              id  matchid_x  player  championid  ss1  ss2  role position  win  \
1239795  1272013     127703       6         103   14    4  SOLO      MID  NaN   

         item1  ...  enemyjunglekills  totcctimedealt  champlvl  pinksbought  \
1239795    NaN  ...               NaN             NaN       NaN          NaN   

         wardsbought  wardsplaced  wardskilled  firstblood  matchid_y  \
1239795          NaN          NaN          NaN         NaN        NaN   

            _merge  
1239795  left_only  

[1 rows x 65 columns]


In [88]:
missing_match_ids = participants[participants['id'].isin(missing_stats['id'])]['matchid'].unique()

In [89]:
print(missing_match_ids)

[127703]


In [90]:
# Filter out matches with missing statistics
matches_filtered = matches[~matches['id'].isin(missing_match_ids)]

In [91]:
matches_filtered.shape

(182527, 8)

In [92]:
stats_combined.shape

(1825279, 57)

In [93]:
matches = matches_filtered

In [94]:
matches.drop(['gameid','queueid','seasonid','creation','version'], axis=1, inplace=True)

In [95]:
teambans.drop(['banturn'], axis=1, inplace=True)

In [96]:
stats_combined.drop(['trinket', 'largestkillingspree', 'largestmultikill', 'killingsprees', 'longesttimespentliving', 'doublekills'], axis=1, inplace=True)

In [97]:
stats_combined.drop(['triplekills','quadrakills','pentakills','legendarykills','largestcrit', 'magicdmgtochamp', 'physdmgtochamp', 'truedmgtochamp'], axis=1, inplace=True)

In [98]:
stats_combined.drop(['totheal','totunitshealed','dmgselfmit','dmgtoobj','dmgtoturrets','timecc','magicdmgtaken','physdmgtaken','truedmgtaken'], axis=1, inplace=True)

In [99]:
stats_combined.drop(['turretkills','totcctimedealt','champlvl','pinksbought','wardsbought','wardsplaced','wardskilled'], axis=1, inplace=True)

In [100]:
stats_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1825279 entries, 0 to 834517
Data columns (total 27 columns):
 #   Column                Dtype
---  ------                -----
 0   id                    int64
 1   win                   int64
 2   item1                 int64
 3   item2                 int64
 4   item3                 int64
 5   item4                 int64
 6   item5                 int64
 7   item6                 int64
 8   kills                 int64
 9   deaths                int64
 10  assists               int64
 11  totdmgdealt           int64
 12  magicdmgdealt         int64
 13  physicaldmgdealt      int64
 14  truedmgdealt          int64
 15  totdmgtochamp         int64
 16  visionscore           int64
 17  totdmgtaken           int64
 18  goldearned            int64
 19  goldspent             int64
 20  inhibkills            int64
 21  totminionskilled      int64
 22  neutralminionskilled  int64
 23  ownjunglekills        int64
 24  enemyjunglekills      int

In [101]:
matches.to_csv("clean_matches.csv", index=False)
participants.to_csv("clean_participants.csv", index=False)
teambans.to_csv("clean_teambans.csv", index=False)
teamstats.to_csv("clean_teamstats.csv", index=False)
stats_combined.to_csv("clean_player_stats.csv", index=False)